# Ratio with a DNN

In this task we will explore the method of measuring ratios of distributions with DNN driven classifications.

## Introduction
We will create data from two Gaussian distributions and measure the ratio of the underlying probability functions in three ways:
- Analytic (only possible in the toy experiment)
- Classic (bin and measure the ratio in each bin)
- Classification (train a DNN to classify between the two datasets)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

## Creating and plotting the data
First we fix the parametrisation of our Gaussian distributions (A and B) and create the data.

In [ ]:
# parametrisation of the underlying probability distributions
loc_a, scale_a = -0.5, 1.0
loc_b, scale_b = 0.5, 1.0

# creating the data
a = np.random.normal(loc=loc_a, scale=scale_a, size=(100000,))
b = np.random.normal(loc=loc_b, scale=scale_b, size=(100000,))

We bin the data in histograms with equidistant bins, plot the histograms and plot (parts of) the raw data.

In [ ]:
# creating the figure
fig = plt.figure()
gs = fig.add_gridspec(2, 1, hspace=0, height_ratios=[3, 1])
(ax1, ax2) = gs.subplots(sharex='col')

# plot histograms
hist_a, bins_a, _ = ax1.hist(a, bins=np.arange(-4, 4.4, 0.4), alpha=0.5, label="Distribution A")
hist_b, bins_b, _ = ax1.hist(b, bins=bins_a, alpha=0.5, label="Distribution B")

# plot 1000 example points
ax2.plot(a[:1000], np.zeros_like(a)[:1000], linestyle="None", marker="|", alpha=0.1)
ax2.plot(b[:1000], np.zeros_like(b)[:1000], linestyle="None", marker="|", alpha=0.1)

# styling plot
ax2.axes.get_yaxis().set_visible(False)
ax2.set_xlabel("x")
ax1.set_ylabel("Counts [#]")
ax2.set_xlim([-4, 4])
ax1.legend()

## DNN based Classification

Now create a DNN model for the classification between Distribution A and Distribution B.
- How many inputs do we have?
- How many outputs do we have?
- How many layers with which activation funcitons do we need?
- Does the network output probabilities or logits?

In [ ]:
# create the model
# TODO
model = tf.keras.Sequential(
    layers=[
        tf.keras.Input(shape=(1,)),
    ]
)

Now compile the model.
- Which loss function do we want?
- Which optimizer do we want?

In [ ]:
# compile the model
optimizer = None  # TODO
loss = None  # TODO
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

Look at the model summary to see how many trainable parameters or model has.

In [ ]:
model.summary()

Now we prepare the data for the training by interleaving datasets A and B and shuffling the data.

In [ ]:
# prepare the data for training (interleave+shuffle)
x = np.concatenate([a, b])
y = np.concatenate([np.ones_like(a), np.zeros_like(b)])
p = np.random.permutation(len(x))
x, y = x[p], y[p]

Now we fit the model to the training data:

In [ ]:
# fit the model
model.fit(x=x, y=y, epochs=5, validation_split=0.2)

## Plotting and Results
In the following cells we will measure the ratio in the above mentioned three ways.

In [ ]:
# define x_values for inference and plotting
x_values = np.arange(-4, 4.01, 0.01)[:, None]

In [ ]:
# analytic solution
def gaussian(x, loc, scale):
    return 1 / np.sqrt(2 * np.pi * scale ** 2) * np.exp(-((x - loc) ** 2) / (2 * scale ** 2))

analytic_ratio = gaussian(x_values, loc=loc_a, scale=scale_a) / gaussian(x_values, loc=loc_b, scale=scale_b)

In [ ]:
# classic solution by histogram division
hist_ratio = hist_a / hist_b

Think about how to transform the network predictions into the ratio.

In [ ]:
# DNN based classification solution
pred = model.predict(x_values)
pred_ratio = None  # TODO

In [ ]:
# plot all the infered ratios
plt.plot(x_values, analytic_ratio, label="Analytic", color="black", linestyle="--")
plt.step(bins_a, np.pad(hist_ratio, (1, 0), 'edge'), label="Histogram Division")
plt.plot(x_values, pred_ratio, label="DNN Based Classification", color="red")

plt.xlabel("x")
plt.ylabel("Ratio A/B")
plt.xlim([-4, 4])
plt.legend()

As you can see, the DNN scale factor has central improvements compared to the histogram division:
- It is continuous over the whole range
- It does not suffer so much from insuficient statistics (towards the edges)

## Summary

This concludes our tutorial for today.

In this tutorial you learned:
- How to measure the ratio of probability distributions:
  - Analytic (only possible in a toy experiment with known probability distributions)
  - Classic (binning and measuring the ratio in each bin)
  - By a DNN driven classification between the distributions
- Why the DNN diven classification approach is favourable compared to the classic approach:
  - Continuous
  - Works with low statistics